# Two-body problem module: Analysis notebook

### Directory structure

#### `README.md` contents
---
Basic usage: 

```sh
python orbits.py CellestialSeer --save_hdf5
```

To show the evolution:

```sh
python orbits.py CellestialSeer \
  --save_hdf5 \
  --gif_name "orbit.gif" \
  --frames 100 \
```

Full options:

```sh
python orbits.py CellestialSeer \
  --save_hdf5 \
  --gif_name "orbit.gif" \
  --frames 100 \
```

---

```bash
celestia/
├── README.md
```

just files or classes and functions too?

### Relativistic vs. classical mechanics

##### (i) Use your module/script to run and show two simulations: one relativistic and one classical for this set of initial conditions. It may be helpful to compare the orbital history in a single plot.

| Parameter | Description                                       | Units      |
|-----------|---------------------------------------------------|------------|
| $e$       | Eccentricity of the orbit                         | $0$        |
| $M$       | Mass of the central black hole                    | $5\times 10^6\,\rm M_\odot$ |
| $a$       | Semi-major axis of the orbit                      | $1\,\rm AU$|
| $N$       | Number of orbital periods to simulate             | $2$        |
| Method    | Numerical method for ODE integration              | RK3        |

In [1]:
# Module import
from orbits import CelestialSeer, ChronoPainter

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Set parameters
M = 5e6 # Solar masses
a = 1.0 # AU
e = 0.0
N = 2.0

In [3]:
# Instantiate the integrator
orbital_sys = CelestialSeer(M, a, e)

In [4]:
# Classical case
t_class, S_class, file_class = orbital_sys.integrate(
    relativistic = False, 
    N = N, 
    method = 'RK3', 
    save_hdf5 = True) 

# Relativistic case
t_relat, S_relat, file_relat = orbital_sys.integrate(
    relativistic = True, 
    N = N, 
    method = 'RK3', 
    save_hdf5 = True)

Saved to outputfolder/M5.0e+06-a1.0-e0.000-class-RK3.h5
Saved to outputfolder/M5.0e+06-a1.0-e0.000-relat-RK3.h5


In [5]:
# Create instance of the painter
painter = ChronoPainter(
    orbits = (file_class, file_relat),
    labels = ('Classical', 'Relativistic'),
    colors = ('khaki', 'magenta')
    )

In [6]:
# # Animate
# painter.paint(
#     gif_name = "rel-vs-class.gif",
#     frames = 75,
#     duration = 0.5,
#     dpi = 100
# )

In [7]:
# Show it
# painter.show_evolution("rel-vs-class.gif")

to ellaborate on: no precession

##### (j) Use the orbital history of both simulations to design a method that quantifies their differences and evaluates the importance of using the relativistic approach for massive objects. Do we need to worry about the relativistic corrections if we replace the black hole with our Sun?


We will see how much the positions and velocities differ *on average* over the entire time evolution. 

In [32]:
def mean_euclidean(s1, s2):
    """
    Calculate the mean Euclidean distance between two sets of positions.
    """
    # Extract positions and velocities
    pos1 = s1[0:2, :]
    pos2 = s2[0:2, :]
    vel1 = s1[2:4, :]
    vel2 = s2[2:4, :]

    # Magnitude of velocities
    v1_mag = np.linalg.norm(vel1, axis = 0) 
    v2_mag = np.linalg.norm(vel2, axis = 0)

    # Calculate the mean distance and velocity
    mean_distance = np.mean(np.linalg.norm(pos1 - pos2, axis = 0)) # AU
    mean_velocity = np.mean(np.abs(v1_mag - v2_mag))               # AU/yr

    return mean_distance, mean_velocity

In [25]:
# Calculate the mean Euclidean distance
mean_distance, mean_velocity = mean_euclidean(S_class, S_relat)
print(f"Mean distance: {mean_distance:.4f} AU")
print(f"Mean velocity: {mean_velocity:.4f} AU/yr")

Mean distance: 1.2848 AU
Mean velocity: 2479.9939 AU/yr


In [8]:
# Sun
M_sun = 1 # Solar masses

# Create another instance
integrator2 = CelestialSeer(M_sun, a, e)

In [9]:
# Classical case
t_class2, S_class2, file_class_sun = integrator2.integrate(
    relativistic = False, 
    N = N, 
    method = 'RK3', 
    save_hdf5 = True)

# Relativistic case
t_relat2, S_relat2, file_relat_sun = integrator2.integrate(
    relativistic = True, 
    N = N, 
    method = 'RK3', 
    save_hdf5 = True)

Saved to outputfolder/M1.0e+00-a1.0-e0.000-class-RK3.h5.
Saved to outputfolder/M1.0e+00-a1.0-e0.000-relat-RK3.h5.


In [10]:
# Create another instance
painter2 = ChronoPainter(
    orbits = (file_class_sun, file_relat_sun),
    labels = ('Classical', 'Relativistic'),
    colors = ('khaki', 'magenta')
    )

In [ ]:
# # Animate
# painter2.paint(
#     gif_name = "rel-vs-class-sun.gif",
#     frames = 75,
#     duration = 0.5,
#     dpi = 100
# )

----------------------------------------------------------
                  STARTING GIF CREATION
----------------------------------------------------------
Saving 75 frames...
----------------------------------------------------------
Progress: 0% (0 images)
Progress: 9% (7 images)
Progress: 19% (14 images)
Progress: 28% (21 images)
Progress: 37% (28 images)
Progress: 47% (35 images)
Progress: 56% (42 images)
Progress: 65% (49 images)
Progress: 75% (56 images)
Progress: 84% (63 images)
Progress: 93% (70 images)
----------------------------------------------------------
All 75 images saved to outputfolder/images.
----------------------------------------------------------
Saving GIF...
----------------------------------------------------------
GIF saved to outputfolder/rel-vs-class-sun.gif.
----------------------------------------------------------
All images have been deleted.
----------------------------------------------------------
                   END OF GIF CREATION
-----------

In [12]:
# Show it
# painter2.show_evolution("rel-vs-class-sun.gif")

short answer: no, we do not need to worry when the mass is that of the sun

### The role of eccentricity (3 points):

##### (k) Use your module/script to run and show three relativistic simulations for objects with different eccentricities, $e$, and assuming the same $M$, $a$, $N$ as above. It may be helpful to compare the orbital history for all values of $e$ in a single plot throughout time.

| Object           | Eccentricity ($e$) | Integration Method |
|------------------|--------------------|--------------------|
| Earth            | 0.01671            | Trapezoidal        |
| Pluto            | 0.25               | Trapezoidal        |
| 7092 Cadmus      | 0.70               | Trapezoidal        |

In [13]:
# Define eccentricities
e_earth  = 0.01671
e_pluto  = 0.25
e_cadmus = 0.70

In [14]:
# Create instances
earth_sys  = CelestialSeer(M, a, e_earth)
pluto_sys  = CelestialSeer(M, a, e_pluto)
cadmus_sys = CelestialSeer(M, a, e_cadmus)

# Integrate
t_earth, S_earth, file_earth = earth_sys.integrate(
    relativistic = True, 
    N = N, 
    method = 'RK3', 
    save_hdf5 = True)

t_pluto, S_pluto, file_pluto = pluto_sys.integrate(
    relativistic = True, 
    N = N, 
    method = 'RK3',
    save_hdf5 = True)

t_cadmus, S_cadmus, file_cadmus = cadmus_sys.integrate(
    relativistic = True, 
    N = N, 
    method = 'RK3', 
    save_hdf5 = True)

Saved to outputfolder/M5.0e+06-a1.0-e0.017-relat-RK3.h5.
Saved to outputfolder/M5.0e+06-a1.0-e0.250-relat-RK3.h5.
Saved to outputfolder/M5.0e+06-a1.0-e0.700-relat-RK3.h5.


In [15]:
# Create instance
painter3 = ChronoPainter(
    orbits = (file_earth, file_pluto, file_cadmus),
    labels = ('Earth', 'Pluto', 'Cadmus'),
    colors = ('khaki', 'magenta', 'white')
    )

In [ ]:
# # Animate
# painter3.paint(
#     gif_name = "diff_eccentricities.gif",
#     frames = 75,
#     duration = 0.5,
#     dpi = 100
# )

----------------------------------------------------------
                  STARTING GIF CREATION
----------------------------------------------------------
Saving 75 frames...
----------------------------------------------------------
Progress: 0% (0 images)
Progress: 9% (7 images)
Progress: 19% (14 images)
Progress: 28% (21 images)
Progress: 37% (28 images)
Progress: 47% (35 images)
Progress: 56% (42 images)
Progress: 65% (49 images)
Progress: 75% (56 images)
Progress: 84% (63 images)
Progress: 93% (70 images)
----------------------------------------------------------
All 75 images saved to outputfolder/images.
----------------------------------------------------------
Saving GIF...
----------------------------------------------------------
GIF saved to outputfolder/diff_eccentricities.gif.
----------------------------------------------------------
All images have been deleted.
----------------------------------------------------------
                   END OF GIF CREATION
--------

In [17]:
# Show it
# painter3.show_evolution("diff_eccentricities.gif")

##### (l) Describe the differences in the orbits of the above objects. What happens to objects with high eccentricities?

to do

### Numerical convergence (3 points):

##### (m) Use your script to generate additional simulations with the same initial conditions as before, but only for $e=0.01671$ (Earth's eccentricity) with RK3, the Trapezoidal method and the higher-order SciPy integrator. Compare the orbital history for all methods in a single plot throughout time.

We are going to use slightly different orbital periods.

In [18]:
# Integrate
t_earth_tpz, S_earth_tpz, file_tpz = earth_sys.integrate(
    relativistic = True, 
    N = 2.01, 
    method = 'TPZ', 
    save_hdf5 = True)

t_earth_spy, S_earth_spy, file_spy = earth_sys.integrate(
    relativistic = True, 
    N = 2.02, 
    method = 'SPY', 
    save_hdf5 = True)

Saved to outputfolder/M5.0e+06-a1.0-e0.017-relat-TPZ.h5.
Saved to outputfolder/M5.0e+06-a1.0-e0.017-relat-SPY.h5.


In [19]:
# Create instance
painter4 = ChronoPainter(
    orbits = (file_earth, file_tpz, file_spy),
    labels = ('RK3', 'TPZ', 'SPY'),
    colors = ('khaki', 'magenta', 'white')
    )

In [ ]:
# # Animate
# painter4.paint(
#     gif_name = "diff_methods.gif",
#     frames = 75,
#     duration = 0.5,
#     dpi = 100
# )

----------------------------------------------------------
                  STARTING GIF CREATION
----------------------------------------------------------
Saving 75 frames...
----------------------------------------------------------
Progress: 0% (0 images)
Progress: 9% (7 images)
Progress: 19% (14 images)
Progress: 28% (21 images)
Progress: 37% (28 images)
Progress: 47% (35 images)
Progress: 56% (42 images)
Progress: 65% (49 images)
Progress: 75% (56 images)
Progress: 84% (63 images)
Progress: 93% (70 images)
----------------------------------------------------------
All 75 images saved to outputfolder/images.
----------------------------------------------------------
Saving GIF...
----------------------------------------------------------
GIF saved to outputfolder/diff_methods.gif.
----------------------------------------------------------
All images have been deleted.
----------------------------------------------------------
                   END OF GIF CREATION
---------------

In [21]:
# Show it
# painter4.show_evolution("diff_methods.gif")

no diff for $N=2$

##### (n) Measure convergence of the simulations with RK3 and Trapezoidal method for $e=0.01671$ by integrating at a number of different time steps. To analyse convergence, you need to define some measure for the error with respect to the higher order method, and then plot it against different time steps for both methods. Thus, you may add additional functions for this to your code in **orbits.py**. 